In [4]:
# 1) Prepara datos (año, pivot por país y estándar de tecnologías)
import pandas as pd

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]
for c in year_cols: df[c] = pd.to_numeric(df[c], errors="coerce")

IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
f = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()
YEAR = [c for c in year_cols if f[c].notna().any()][-1]

tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
f["Technology_std"] = f["Technology"].map(tech_map).fillna(f["Technology"])

# Pivot por país (1 punto = país) para el scatter
idx = pd.MultiIndex.from_product([f["Country"].dropna().unique(),
                                  sorted(f["Technology_std"].dropna().unique())],
                                 names=["Country","Technology_std"])
grid = (f.groupby(["Country","Technology_std"])[YEAR].sum(min_count=1)
          .reindex(idx, fill_value=0.0).reset_index())
piv = (grid.pivot(index="Country", columns="Technology_std", values=YEAR)
          .fillna(0.0).reset_index())

for c in ["Solar","Wind","Hydro","Bio","Fossil"]:
    if c not in piv.columns: piv[c] = 0.0
piv["Total"] = piv.drop(columns=["Country"]).sum(axis=1, numeric_only=True)
piv["DominantTech"] = piv[["Solar","Wind","Hydro","Bio","Fossil"]].idxmax(axis=1)

# Para que no se amontonen, quitamos (0,0) solo del SCATTER:
piv_viz = piv[(piv["Solar"]>0) | (piv["Wind"]>0)].copy()
print(f"Año: {YEAR} | países con Solar/Wind>0: {len(piv_viz)}  | filas totales CSV: {len(f)}")

Año: F2023 | países con Solar/Wind>0: 82  | filas totales CSV: 570


In [ ]:
# OG DATASET --- Prepare Renewable Energy dataset for ScatterBrush ---
import pandas as pd
import re

# 1) Load + filter valid ISO2
csv_path = "Renewable_Energy.csv"
f = pd.read_csv(csv_path)
iso2_clean = f["ISO2"].astype(str).str.strip()
f = f[iso2_clean.ne("") & f["ISO2"].notna()].copy()

# 2) Detect available year columns dynamically
year_cols = [c for c in f.columns if re.fullmatch(r"F\d{4}", c)]
year_cols = sorted(year_cols)

# 3) Standardize Technology names before combining
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
f["Technology_std"] = f["Technology"].map(tech_map).fillna(f["Technology"])

# Extract MW/GWh abbreviation
def extract_abbr(unit: str):
    if not isinstance(unit, str):
        return None
    m = re.search(r"\b(MW|GWh)\b", unit, flags=re.IGNORECASE)
    return m.group(1) if m else None

f["UnitAbbr"] = f["Unit"].apply(extract_abbr)
f = f[f["UnitAbbr"].notna()].copy()

# 4) Create TechUnit using standardized names + abbrev
f["TechUnit"] = f["Technology_std"].astype(str).str.strip() + " (" + f["UnitAbbr"] + ")"

# Convert year columns to numeric and drop rows with no data
for c in year_cols:
    f[c] = pd.to_numeric(f[c], errors="coerce")
f = f.dropna(subset=year_cols, how="all")

# 5) Keep only relevant columns
keep_cols = ["ObjectId", "Country", "TechUnit"] + year_cols
df_scatter = f.loc[:, keep_cols].reset_index(drop=True)

# 6) Auto-detect TechUnit options for the scatter widget
tech_options = sorted(df_scatter["TechUnit"].unique().tolist())

# 7) Create XY_var* kwargs automatically
xy_kwargs = {f"XY_var{i+1}": t for i, t in enumerate(tech_options)}

# Optional: get min/max year for slider
year_min = min(int(c[1:]) for c in year_cols)
year_max = max(int(c[1:]) for c in year_cols)

print(f"TechUnits detected: {len(tech_options)} → {tech_options[:5]}...")
print(f"Years available: {year_min}–{year_max}")
df_scatter.head()

In [ ]:
# OG DATASET --- Build WIDE Energy table: ObjectId; Country; Continent; <TechUnit>__FYYYY for all years ---

import re
import pandas as pd

# 1) Load + ISO2 filter
csv_path = "Renewable_Energy.csv"
f = pd.read_csv(csv_path)
iso2_clean = f["ISO2"].astype(str).str.strip()
f = f[iso2_clean.ne("") & f["ISO2"].notna()].copy()

# 2) Detect FYYYY year columns
year_cols = sorted([c for c in f.columns if re.fullmatch(r"F\d{4}", c)])
for c in year_cols:
    f[c] = pd.to_numeric(f[c], errors="coerce")

# 3) Standardize Technology + extract MW/GWh
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
f["Technology_std"] = f["Technology"].map(tech_map).fillna(f["Technology"])

def extract_abbr(unit: str):
    if not isinstance(unit, str):
        return None
    m = re.search(r"\b(MW|GWh)\b", unit, flags=re.IGNORECASE)
    return m.group(1).upper() if m else None

f["UnitAbbr"] = f["Unit"].apply(extract_abbr)
f = f[f["UnitAbbr"].notna()].copy()

# 4) TechUnit = "<Tech> (<MW|GWh>)"
f["TechUnit"] = f["Technology_std"].astype(str).str.strip() + " (" + f["UnitAbbr"] + ")"

# 5) Continent from ISO2 using libraries (comprehensive)

def add_continent_from_iso2(df: pd.DataFrame) -> pd.DataFrame:
    s = df["ISO2"].astype(str).str.strip().str.upper().replace({"": None, "NA": None, "NAN": None})

    # Preferred: country_converter
    try:
        import country_converter as coco
        cc = coco.CountryConverter()
        # coco returns a list; convert back to a Series aligned to index
        cont_list = cc.convert(names=s.tolist(), src="ISO2", to="continent", not_found=None)
        cont_series = pd.Series(cont_list, index=df.index)
        df["Continent"] = cont_series.fillna("Unknown")
        return df
    except Exception:
        pass

    # Fallback: pycountry_convert
    try:
        from pycountry_convert import (
            country_alpha2_to_continent_code as a2_to_cc,
            convert_continent_code_to_continent_name as cc_to_name,
        )
        def conv(a2):
            try:
                return cc_to_name(a2_to_cc(a2))
            except Exception:
                return None
        cont_series = s.map(conv).fillna("Unknown")
        df["Continent"] = cont_series
        return df
    except Exception:
        # Last resort: mark unknown (so pipeline doesn’t break)
        df["Continent"] = "Unknown"
        return df

# Apply
f = add_continent_from_iso2(f)

# 6) Ensure ObjectId exists (if not in CSV, synthesize)
if "ObjectId" not in f.columns:
    f["ObjectId"] = range(1, len(f) + 1)

# 7) Build WIDE table: index = (ObjectId, Country, Continent); columns = "<TechUnit>__FYYYY"
#   a) aggregate per country+techunit (sum over duplicates)
agg = f.groupby(["Country","Continent","TechUnit"], as_index=False)[year_cols].sum()    #! DEBUG REMOVED: "ObjectId",
#   b) long -> add combined column name "<TechUnit>__FYYYY"
long = agg.melt(id_vars=["Country","Continent","TechUnit"],                             #! DEBUG REMOVED: "ObjectId",
                value_vars=year_cols, var_name="YearCol", value_name="Value")
long["col"] = long["TechUnit"] + "__" + long["YearCol"]
#   c) wide pivot
wide = (long.pivot_table(index=["Country","Continent"], columns="col",                  #! DEBUG REMOVED: "ObjectId",
                         values="Value", aggfunc="sum")
            .reset_index()
            .fillna(0.0))

# 8) Output objects you’ll use later
df_wide = wide  # columns: ObjectId, Country, Continent, <TechUnit>__FYYYY, ...
tech_options = sorted(f["TechUnit"].unique().tolist())  # all TechUnits (MW+GWh)
xy_kwargs = {f"XY_var{i+1}": t for i, t in enumerate(tech_options)}  # for widget call

print(f"Wide shape: {df_wide.shape}  | TechUnits: {len(tech_options)}  | Years: {len(year_cols)}")

# Consistent color palette for continents
continent_colors = {
    "Asia": "#1f77b4",
    "Europe": "#ff7f0e",
    "Africa": "#2ca02c",
    "Oceania": "#d62728",
    "America": "#9467bd",
}

df_wide

In [ ]:
from Isea.scatter import ScatterBrush
import re

# Year range
years_int = sorted({int(m.group(1)) for c in df_wide.columns
                    for m in [re.search(r"__F(\d{4})$", str(c))] if m})
year_min, year_max = years_int[0], years_int[-1]


xy_kwargs = {f"XY_var{i+1}": t for i, t in enumerate(tech_options, start=1)}

rows = df_wide.to_dict("records")   # full table; widget remaps per year

w_scatter = ScatterBrush(
    data=rows,
    x=tech_options[1], y=tech_options[3],
    color="Continent",
    colorMap=continent_colors,
    label="Country",      # ✅ display name
    key="Country",        # ✅ stable unique id (one row per country now)
    width=1200, height=500,
    panel_position="right", panel_width=320, panel_height=220,
    YearMin=year_min, YearMax=year_max,
    **xy_kwargs,
)
display(w_scatter)

In [ ]:
# --- Minimal linked view: second scatter shows only the selected countries ---

import pandas as pd
from Isea.scatter import ScatterBrush

# Pull current axis/fields from the first widget (robust to either attr or .options)
opts = getattr(w_scatter, "options", {}) or {}
x_col = getattr(w_scatter, "x", None) or opts.get("x")
y_col = getattr(w_scatter, "y", None) or opts.get("y")
label_col = getattr(w_scatter, "label", None) or opts.get("label") or "Country"
color_col = getattr(w_scatter, "color", None) or opts.get("color") or "Continent"
key_col   = getattr(w_scatter, "key", None)   or opts.get("key")   or "Country"


# print(opts, x_col, y_col, label_col, color_col, key_col)
# XY_var* and year range from earlier cells
xy_kwargs = {f"XY_var{i+1}": t for i, t in enumerate(tech_options, start=1)}

# Empty second widget (data will be filled on first selection)
w_scatter_sel = ScatterBrush(
    data=pd.DataFrame([], columns=[label_col, color_col, key_col]).to_dict("records"),
    x=x_col, y=y_col,
    color=color_col,colorMap=continent_colors, 
    label=label_col, key=key_col,
    width=900, height=450, panel_position="right", panel_height=160,
    # title="Linked view (selection from first scatter)",
    YearMin=year_min, YearMax=year_max,
    **xy_kwargs,
)
display(w_scatter_sel)

# Keep a df for downstream cells
df_selected = pd.DataFrame(columns=df_wide.columns)


def _link_selection_to_second(change):
    global df_selected
    sel = change.get("new") or {}
    rows = sel.get("rows") or []

    # countries from the selection (row has Country or fallback to label/key)
    countries = [r.get("Country") or r.get(label_col) or r.get(key_col) for r in rows]
    countries = [c for c in countries if isinstance(c, str)]

    # Update df_selected (full wide rows so year/XY remain interactive)
    if countries:
        df_selected = df_wide[df_wide["Country"].isin(countries)].copy()
    else:
        df_selected = pd.DataFrame(columns=df_wide.columns)

    # Keep linked widget in sync:
    #  - data: full rows (so its slider & XY pills work)
    #  - axes: mirror current axes from the first widget
    cur_x = getattr(w_scatter, "x", None) or opts.get("x")
    cur_y = getattr(w_scatter, "y", None) or opts.get("y")
    w_scatter_sel.data = (
        df_selected.assign(key=lambda d: d["Country"], label=lambda d: d["Country"])
                   .to_dict("records")
    )
    if cur_x and cur_y:
        w_scatter_sel.x = cur_x
        w_scatter_sel.y = cur_y

    # optional: clear selection inside the second chart each update
    w_scatter_sel.selection = {"type": None, "keys": [], "rows": [], "epoch": int(__import__("time").time()*1000)}

# Wire first -> second (live updates)
w_scatter.observe(_link_selection_to_second, names="selection")

In [ ]:
import pandas as pd
from Isea import ParallelEnergy

df = pd.read_csv("Isea/Energy_clean.csv")

years = [c for c in df.columns if c.startswith("F")]
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])


In [ ]:
w = ParallelEnergy(
    df, years,
    tech_col="Technology_std",
    label_col="Country",
    dims=("Fossil", "Solar", "Hydro", "Wind", "Bio"),
    year_start="F2023",
    width=1100, height=560,
    margin=dict(left=50, right=180, top=80, bottom=36),  # ajusta si quieres
    panel_position="bottom",       # "right" o "bottom"
    panel_width=320, panel_height=220,
    log_axes=False, normalize=False, reorder=True
)
w


ParallelEnergy(data={'years': ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008'…

In [11]:
# w.show_selection()

""


In [12]:
from IPython.display import display, clear_output
import ipywidgets as ipw

out = ipw.Output()
display(out)

def on_sel(change):
    with out:
        clear_output(wait=True)
        df_sel = w.selection_df()
        print(f"point selection — {len(df_sel)} points")
        display(df_sel)

# Escucha cambios del widget
w.observe(on_sel, "selection")


Output()

In [13]:
# w2 = w.new_from_selection(
#     width=1000, height=520,
#     margin=dict(l=48, r=160, t=72, b=32),
#     panel_position="bottom",
#     panel_height=200
# )
# w2


ValueError: No hay selección (keys vacío).

In [14]:
import inspect
from Isea.radial_stacked_bar import RadialStackedBar

# Ver la firma de la función
print(inspect.signature(RadialStackedBar.__init__))

# Ver si year_cols está en los parámetros
params = inspect.signature(RadialStackedBar.__init__).parameters
print("year_cols en parámetros:", "year_cols" in params)

(self, df: pandas.core.frame.DataFrame, group_col: str, category_col: str, year_cols: Optional[List[str]] = None, year_start: Optional[str] = None, agg: str = 'sum', width: int = 900, height: int = 900, inner_radius: int = 200, pad_angle: float = 0.015, color_scheme: str = 'schemeSpectral', sort_on_click: bool = True, title: Optional[str] = None, custom_colors: Optional[List[str]] = None, **kwargs)
year_cols en parámetros: True


In [15]:
# ========== PRIMERO: Preparar datos (IGUAL A LA CELDA 1) ==========
import pandas as pd
from Isea import RadialStackedBar

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]

# Convertir a numéricas
for c in year_cols: 
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Filtrar por indicador y unidad
IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
df = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()

# IMPORTANTE: Estandarizar tecnologías
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])

print(f"✅ Datos preparados: {len(df)} filas")
print(f"✅ Columnas: {df.columns.tolist()}")
print(f"✅ Tecnologías únicas: {df['Technology_std'].unique()}")

# ========== AHORA: Crear RadialStackedBar ==========
radial = RadialStackedBar(
    df=df,
    group_col="Country",
    category_col="Technology_std",
    year_cols=year_cols,
    year_start="F2023",
    width=900,
    height=900,
    inner_radius=200,
    title="⚡ Capacidad Instalada",
    custom_colors=["#d73027","#f46d43","#fdae61","#fee08b","#e6f598","#abdda4","#66c2a5","#3288bd"]
)

display(radial)
print("✅ Widget creado exitosamente!")

✅ Datos preparados: 570 filas
✅ Columnas: ['ObjectId', 'Country', 'ISO2', 'ISO3', 'Indicator', 'Technology', 'Energy_Type', 'Unit', 'Source', 'CTS_Name', 'CTS_Code', 'CTS_Full_Descriptor', 'F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008', 'F2009', 'F2010', 'F2011', 'F2012', 'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020', 'F2021', 'F2022', 'F2023', 'Technology_std']
✅ Tecnologías únicas: ['Fossil' 'Hydro' 'Bio' 'Solar' 'Wind']


RadialStackedBar(data={'years': ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F200…

✅ Widget creado exitosamente!


In [16]:
print("Columns:", df.columns.tolist())
print("Has Technology_std:", "Technology_std" in df.columns)
print("Year cols:", year_cols)
print("Sample data:\n", df[["Country", "Technology_std", year_cols[0]]].head())

Columns: ['ObjectId', 'Country', 'ISO2', 'ISO3', 'Indicator', 'Technology', 'Energy_Type', 'Unit', 'Source', 'CTS_Name', 'CTS_Code', 'CTS_Full_Descriptor', 'F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008', 'F2009', 'F2010', 'F2011', 'F2012', 'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020', 'F2021', 'F2022', 'F2023', 'Technology_std']
Has Technology_std: True
Year cols: ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008', 'F2009', 'F2010', 'F2011', 'F2012', 'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020', 'F2021', 'F2022', 'F2023']
Sample data:
                         Country Technology_std     F2000
0  Afghanistan, Islamic Rep. of         Fossil    29.725
1  Afghanistan, Islamic Rep. of          Hydro   191.503
2                       Albania          Hydro  1453.000
3                       Algeria         Fossil  5846.210
4                       Algeria          Hydro   276.600


In [17]:
# Fuerza recarga
import sys
for mod in list(sys.modules.keys()):
    if 'Isea' in mod:
        del sys.modules[mod]

# Reimporta
from Isea import RadialStackedBar
from IPython.display import display

# Preparar datos (mismo que antes)
import pandas as pd

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]

for c in year_cols: 
    df[c] = pd.to_numeric(df[c], errors="coerce")

IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
df = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()

tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])

-1

F2023
Country
ISO3
Technology_std
right
✅ WorldRenewable widget created successfully!
selection
print("_esm path:", RadialStackedBar._esm)
print("Existe:", Path(RadialStackedBar._esm).exists())

# Si no existe, dinos y crearemos el archivo
print("\nContenido esperado del archivo:")
import inspect
print(inspect.getfile(RadialStackedBar))

SyntaxError: invalid character '✅' (U+2705) (556875360.py, line 39)

In [18]:
# ========== PREPARACIÓN AGRESIVA DE DATOS ==========
import pandas as pd
from Isea import RadialStackedBar

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]

# Convertir a numéricas
for c in year_cols: 
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Filtrar por indicador y unidad
IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
df = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()

# Estandarizar tecnologías
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])

# ========== Filtrar solo tecnologías estándar ==========
df_clean = df[
    df["Technology_std"].isin(["Fossil", "Solar", "Wind", "Hydro", "Bio"])
].copy()

# ========== NUEVO: Agrupar por país y tecnología ==========
df_grouped = df_clean.groupby(["Country", "Technology_std"])[year_cols].sum().reset_index()

# ========== NUEVO: Filtrar países con TODAS las tecnologías ==========
# Contar cuántas tecnologías tiene cada país
tech_count = df_grouped.groupby("Country")["Technology_std"].nunique()

# Mantener solo países con LAS 5 TECNOLOGÍAS
required_techs = 5  # Fossil, Solar, Wind, Hydro, Bio
complete_countries = tech_count[tech_count == required_techs].index.tolist()

print(f"📊 Países totales: {df_grouped['Country'].nunique()}")
print(f"✅ Países con 5 tecnologías: {len(complete_countries)}")

# Filtrar
df_grouped = df_grouped[df_grouped["Country"].isin(complete_countries)].copy()

# ========== NUEVO: Eliminar filas donde TODOS los años son 0 ==========
df_grouped["Total"] = df_grouped[year_cols].sum(axis=1)
df_grouped = df_grouped[df_grouped["Total"] > 0].drop(columns=["Total"])

# ========== NUEVO: Filtrar por año más reciente con datos ==========
# Solo mantener países con datos en F2023 (año más reciente)
last_year = "F2023"
df_grouped = df_grouped[df_grouped[last_year] > 0].copy()

print(f"✅ Datos finales después de filtrar: {len(df_grouped)} filas")
print(f"✅ Países únicos: {df_grouped['Country'].nunique()}")
print(f"✅ Tecnologías: {df_grouped['Technology_std'].unique()}")

# ========== Crear RadialStackedBar ==========
radial = RadialStackedBar(
    df=df_grouped,
    group_col="Country",
    category_col="Technology_std",
    year_cols=year_cols,
    year_start="F2023",
    width=900,
    height=900,
    inner_radius=200,
    title="⚡ Capacidad Instalada (MW) - Países Completos",
    custom_colors=["#d73027","#f46d43","#fdae61","#fee08b","#e6f598","#abdda4","#66c2a5","#3288bd"]
)

display(radial)
print("✅ Widget creado con datos limpios!")

📊 Países totales: 216
✅ Países con 5 tecnologías: 26
✅ Datos finales después de filtrar: 130 filas
✅ Países únicos: 26
✅ Tecnologías: ['Bio' 'Fossil' 'Hydro' 'Solar' 'Wind']


RadialStackedBar(data={'years': ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F200…

✅ Widget creado con datos limpios!


In [19]:
# ========== WorldRenewable Widget (AnyWidget Version) ==========
from Isea import WorldRenewable
from IPython.display import display
import pandas as pd

# Prepare data with Technology_std column
df_world = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df_world.columns if c.startswith("F")]
for c in year_cols: 
    df_world[c] = pd.to_numeric(df_world[c], errors="coerce")

# Add Technology_std mapping
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df_world["Technology_std"] = df_world["Technology"].map(tech_map).fillna(df_world["Technology"])

# Create the widget
world_widget = WorldRenewable(
    df=df_world,
    year_cols=year_cols,
    country_col="Country",
    iso3_col="ISO3",
    tech_col="Technology_std",
    start_year="F2023",  # Start at the most recent year
    width=1200,
    height=660,
    normalize=False
)

display(world_widget)

# Add observer to track selection
def on_world_selection(change):
    sel = change["new"]
    if sel and sel.get("iso3"):
        year = sel.get('year', 'N/A')
        value = sel.get('value')
        value_str = f"{value:.1%}" if value is not None else "N/A"
        print(f"Selected: {sel['name']} ({sel['iso3']}) - Year {year} - Renewable: {value_str}")

world_widget.observe(on_world_selection, names="selection")

WorldRenewable(data={'years': ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008'…

✅ WorldRenewable widget created successfully!


Final code parallel

In [5]:
import importlib, Isea
importlib.reload(Isea)

import pandas as pd
from Isea import EnergyQuad

df = pd.read_csv("Isea/Energy_clean.csv")

years = [c for c in df.columns if c.startswith("F")]
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])

dash = EnergyQuad(
    df, years,
    tech_col="Technology_std", label_col="Country",
    dims=("Fossil","Solar","Hydro","Wind","Bio"),
    year_start="F2023",
    width=1180, left_width=720,   # balancea columnas
    left_height=460, table_height=190,
    right_width=440, insight_height=230, mini_height=260,
    log_axes=False, normalize=False, reorder=True
)
dash



EnergyQuad(data={'years': ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008', 'F…

In [24]:
dash.selection_df()


,Country,Year,Fossil,Solar,Hydro,Wind,Bio,DominantTech
0,Brazil,F2023,30205.946,0.000,109903.366,29135.122,17596.939,Hydro
1,Canada,F2023,35176.900,0.000,83138.000,16989.418,2699.857,Hydro
2,India,F2023,0.000,73109.342,47335.995,44736.240,10752.414,Solar
3,Japan,F2023,0.000,89077.000,28129.000,5343.000,5805.000,Solar
4,Norway,F2023,602.674,665.764,34401.000,5065.000,78.950,Hydro
5,Russian Federation,F2023,0.000,0.000,50571.000,2517.750,1372.500,Hydro
6,"Türkiye, Rep. of",F2023,47420.199,11292.630,31779.490,11697.160,2001.067,Fossil
7,United States,F2023,0.000,139205.277,84316.753,148019.861,10989.500,Wind
